In [1]:
import numpy as np
from scipy import optimize, special
import itertools
from tqdm import trange
from scipy.special import expit


## Initialize weights for theta values

In [2]:
def rand_initialize_weights(L_in, L_out):
    '''Returns a vector of dimension: (L_out, L_in + 1) of random values'''
    t = np.zeros((L_out, L_in + 1))
    e = 0.12 # sigma
    t[:] = np.random.randn(*t.shape) * 2.0 * e - e
    return t

In [3]:
# Load data
data = np.genfromtxt('./data/tt.csv', delimiter=',')
y = data[:,0].reshape(-1,1)  # Shape: (688,1)
X = data[:, 1:]              # Shape: (m, input_layer_size)
m = len(y)

input_layer_size = 784
hidden_layer_size = 120      # hidden layer has 120 nodes excluding bias 
num_labels = 26              # 26 total output values: a=0, b=1, ... , z=25

# Initialize theta values
# theta1 size = (input_layer_size + 1, hidden_layer_size)
# theta2 size = (hidden_layer_size + 1, num_labels)
np.random.seed(1999)
theta1 = rand_initialize_weights(input_layer_size, hidden_layer_size)
theta2 = rand_initialize_weights(hidden_layer_size, num_labels)

print(f'theta1 shape: {np.shape(theta1)}, theta2 shape: {np.shape(theta2)}')

lam = 0.0 # Lambda used for regularization in cost function

theta1 shape: (120, 785), theta2 shape: (26, 121)


## Sigmoid function and derivative
* Used for activation function in forward propogation
* Derivative used for back propogation

In [4]:
def sigmoid(z):
   # return (1 / (1 + np.exp(-z)))
    return expit(z)

def dx_sigmoid(z):
    res = sigmoid(z)
    return (res * (1 - res))

print(sigmoid(0))
print(dx_sigmoid(0))

0.5
0.25


In [5]:
def return_labeled(y, num_labels):
    '''returns (m,num_labels) matrix where each column corresoponds to an example
    all indexes are 0, each column contains a single 1, the row corresponding to the answer'''
    out = np.zeros((m, num_labels))
    for row in range(0, m):
        label = int(y[row])
        out[row, label] = 1
    return out

In [6]:
def forward_backward(X,y):
    y_k = return_labeled(y, num_labels) # output matrix used for cost evaluation
    # Forward propogation
    a1 = np.c_[np.ones((m,1)), X]   # layer 1 : adding a bias column of 1's to X. (m, input_size + 1)
    z2 = a1.dot(theta1.T)           # layer 2 matrix calculation
    a2 = sigmoid(z2)                # activation of layer 2: shape(m, hidden_layer_size)
    a2 = np.c_[np.ones((m, 1)), a2] # adding a bias column of 1's : shape(m, hidden_layer_size + 1)
    z3 = a2.dot(theta2.T)           # layer3 || output layer calculation
    a3 = sigmoid(z3)                # activation of output layer: size = (m, num_labels)
    
    # Cost function
    # Check this after training
    inner_term0 = (-y_k * np.log(a3))
    inner_term1 = (1 - y_k) * np.log(1 - a3)
    left_side = np.sum(inner_term0 + inner_term1) / m
    right_side = np.sum(theta1[:, 1:] ** 2) + np.sum(theta2[:,1:] ** 2) # sum of all theta vals squred excluding theta index0 
    right_side = (lam / 2 / m) * right_side
    cost = left_side + right_side
    print(f'cost: {cost}')
    
    # Back propogation
    delta3 = a3 - y_k
    delta2 = delta3.dot(theta2)[:,1:] * dx_sigmoid(z2) # excluding bias
    Gradient1 = (delta2.T).dot(a1)
    Gradient2 = (delta3.T).dot(a2)
    

    return cost, a3, Gradient1, Gradient2


In [9]:

learning_rate = 0.001
batch_size = 128

# reinitialize thetas
theta1 = rand_initialize_weights(input_layer_size, hidden_layer_size)
theta2 = rand_initialize_weights(hidden_layer_size, num_labels)

losses, accuracies = [], []

for i in (t := trange(m)):
    J, h_x, grad1, grad2 = forward_backward(X,y)
    prediction = np.argmax(h_x, axis=1)
    accuracy = (prediction == y[i]).mean()
    
    # Gradient descent:
    theta1 = theta1 - (learning_rate * grad1)
    theta2 = theta2 - (learning_rate * grad2)
    
    loss = J.mean()
    losses.append(loss)
    
    accuracies.append(accuracy)
    t.set_description("loss %.2f accuracy %.2f" % (loss, accuracy))

    

loss 0.02 accuracy 0.00:   1%|▏         | 10/688 [00:00<00:07, 94.17it/s] 

cost: -16.002664998057345
cost: -12.360678857005533
cost: -9.75562748363745
cost: -7.851756219928978
cost: -6.422448127420966
cost: -5.320138192476938
cost: -4.448776895102651
cost: -3.7448211527121726
cost: -3.165260711356478
cost: -2.6802605042385563
cost: -2.2686238083357506
cost: -1.9149505645770368
cost: -1.6078204532259386
cost: -1.3386052963523525
cost: -1.1006756722873448
cost: -0.8888597114472543
cost: -0.6990666182095243
cost: -0.5280199736675979
cost: -0.37306560200146677
cost: -0.23203098802959587
cost: -0.1031209303652523
cost: 0.015160940471411838
cost: 0.12407191590874236


loss 1.54 accuracy 0.00:   6%|▌         | 38/688 [00:00<00:05, 112.65it/s]

cost: 0.22467622160820647
cost: 0.3178808217784276
cost: 0.40446355287026853
cost: 0.48509543127077953
cost: 0.5603584915828704
cost: 0.630760164011506
cost: 0.6967449482550201
cost: 0.7587039581457762
cost: 0.8169827763322659
cost: 0.8718879578987018
cost: 0.9236924464498333
cost: 0.9726401091224927
cost: 1.0189495534259252
cost: 1.062817355307733
cost: 1.1044208018828796
cost: 1.1439202320112585
cost: 1.1814610420062863
cost: 1.2171754111910844
cost: 1.2511837920307807
cost: 1.2835962015855669
cost: 1.3145133446128034
cost: 1.344027593463329
cost: 1.3722238457096003
cost: 1.399180277011584
cost: 1.424969003914864
cost: 1.4496566689620722
cost: 1.473304958587424
cost: 1.4959710626786977
cost: 1.517708083370779
cost: 1.5385653995316582
cost: 1.5585889924763725


loss 1.91 accuracy 0.00:  10%|█         | 69/688 [00:00<00:04, 128.64it/s]

...

cost: 2.1694307893420923
cost: 2.1694390201484213
cost: 2.169447222579784
cost: 2.1694553967933667
cost: 2.1694635429450484
cost: 2.1694716611894056
cost: 2.1694797516797313
cost: 2.1694878145680527
cost: 2.169495850005143
cost: 2.1695038581405366
cost: 2.169511839122546
cost: 2.169519793098272
cost: 2.169527720213622
cost: 2.16953562061332
cost: 2.169543494440924
cost: 2.169551341838836
cost: 2.1695591629483184
cost: 2.169566957909502
cost: 2.169574726861407
cost: 2.1695824699419464
cost: 2.1695901872879473
cost: 2.1695978790351536
cost: 2.1696055453182486
cost: 2.1696131862708583
cost: 2.169620802025567
cost: 2.16962839271393
cost: 2.1696359584664804
cost: 2.169643499412746
cost: 2.1696510156812576
cost: 2.169658507399558


loss 2.17 accuracy 0.00:  95%|█████████▌| 656/688 [00:04<00:00, 150.95it/s]

cost: 2.169665974694216
cost: 2.1696734176908365
cost: 2.169680836514068
cost: 2.169688231287618
cost: 2.1696956021342566
cost: 2.169702949175835
cost: 2.1697102725332864
cost: 2.1697175723266424
cost: 2.1697248486750382
cost: 2.1697321016967264
cost: 2.169739331509081
cost: 2.1697465382286123
cost: 2.16975372197097
cost: 2.16976088285096
cost: 2.169768020982543
cost: 2.1697751364788544
cost: 2.1697822294522013
cost: 2.1697893000140813
cost: 2.169796348275185
cost: 2.1698033743454075
cost: 2.16981037833385
cost: 2.1698173603488375
cost: 2.169824320497918
cost: 2.1698312588878776
cost: 2.169838175624742
cost: 2.1698450708137873
cost: 2.1698519445595466
cost: 2.1698587969658196
cost: 2.1698656281356747
cost: 2.1698724381714634
cost: 2.16987922717482


loss 2.17 accuracy 0.00: 100%|██████████| 688/688 [00:04<00:00, 147.08it/s]

cost: 2.1698859952466734
cost: 2.1698927424872534
cost: 2.1698994689960958
cost: 2.1699061748720516
cost: 2.1699128602132896
cost: 2.1699195251173076
cost: 2.1699261696809384
cost: 2.16993279400035
cost: 2.1699393981710613
cost: 2.1699459822879397
cost: 2.1699525464452147
cost: 2.169959090736478
cost: 2.169965615254693
cost: 2.1699721200921998
cost: 2.1699786053407193
cost: 2.169985071091362
cost: 2.1699915174346334
cost: 2.1699979444604356
cost: 2.1700043522580774
cost: 2.1700107409162785
cost: 2.1700171105231743
cost: 2.1700234611663203
cost: 2.170029792932701
cost: 2.170036105908731
cost: 2.1700424001802627
cost: 2.170048675832589
cost: 2.1700549329504524
cost: 2.1700611716180465


In [8]:
# Model training
